In [9]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.dates as dates
import matplotlib.pyplot as plt

In [10]:
station_metadata = pd.read_csv("Rainfall_database_Metadata.csv")
station_metadata = station_metadata[station_metadata['Site_name'] != 'Poloa']
station_metadata.drop(columns=['Source', 'Time Resolution'], inplace=True)

# Rainfall Data
datasamples_rainfall = []
monthly_rainfall = []

In [11]:
station_name_index_rainfall = {}

#get data, resample all to monthly
for i in range(len(station_metadata['Site_name'])):    
    # Check and fix if airport data
    if station_metadata.iloc[i]['Site_name'] == 'Airport':
        datasamples_rainfall.append(pd.read_csv(station_metadata.iloc[i]['Filename']))
        datasamples_rainfall[i]['RNF_in'].replace(to_replace=' ', value=np.nan, inplace=True)
        datasamples_rainfall[i]['RNF_in'] = datasamples_rainfall[i]['RNF_in'].astype(float)
    else:
        datasamples_rainfall.append(pd.read_csv(station_metadata.iloc[i]['Filename'], index_col=0))
            
    datasamples_rainfall[i].fillna(0, inplace=True)
    station_name_index_rainfall[station_metadata.iloc[i]['Site_name']] = i
    datasamples_rainfall[i]["DateTime"] = pd.to_datetime(datasamples_rainfall[i]["DateTime"])
    monthly_rainfall.append(datasamples_rainfall[i].resample('M', on='DateTime', origin='start').mean())
    monthly_rainfall[i].dropna(inplace=True)
    if station_metadata.iloc[i]['Site_name'] == 'Aasu' or station_metadata.iloc[i]['Site_name'] == 'Afono':
        monthly_rainfall[i].drop(index=monthly_rainfall[i].index[0], axis=0, inplace=True)

In [12]:
station_name_file = open('DataFileNames.txt', 'w')
for i in range(len(monthly_rainfall)):
    name = str(list(station_name_index_rainfall.keys())[i])
    station_name_file.write(name+'\n')
    column_file = open(name+'_column_names.txt', 'w')
    temp_columns = monthly_rainfall[i].columns[:]
    for column in temp_columns:
        x_dates = monthly_rainfall[i].index
        x_num = dates.date2num(x_dates)
        trend = np.polyfit(x_num, monthly_rainfall[i][column], deg=1)
        fitting_function = np.poly1d(trend)
        monthly_rainfall[i][str(column)+'_Linear_fit'] = fitting_function(x_num)
        column_file.write(str(column)+'\n')
        monthly_rainfall[i].to_csv(name+'_'+str(column)+'_processed.csv', index=True, columns=[column, str(column)+'_Linear_fit'])
    column_file.close()
station_name_file.close()